In [1]:
exec('from __future__ import unicode_literals')

import os
import sys
import random

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import repeat

from onmt.utils.logging import init_logger
from onmt.utils.misc import split_corpus
from onmt.translate.translator import build_translator

import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
from kp_gen_eval import _get_parser


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from string import punctuation
import onmt.keyphrase.pke as pke
from nltk.corpus import stopwords

import onmt.keyphrase.kp_inference as kp_inference

/Users/rui.meng/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [6]:
import importlib
importlib.reload(kp_inference)

<module 'onmt.keyphrase.kp_inference' from '/Users/rui.meng/project/OpenNMT-kpg/onmt/keyphrase/kp_inference.py'>

### Load a paragraph from 10-K docs (assume current directory is OpenNMT-kpg/notebook/)

In [4]:
data_root_path = '../data/salesforce/10k'
print(os.path.abspath(data_root_path))
doc_dicts = []
for subdir, dirs, files in os.walk(data_root_path):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".txt"):
#             print(filepath)
            text = open(filepath, 'r').readlines()
            pars = [l for l in text if len(l.split()) > 20]
            doc = {'name': file, 'path': filepath, 'text': text, 'paragraph': pars}
#             print('#line=%d, #par=%d' % (len(text), len(pars)))
            doc_dicts.append(doc)
    
print('Loaded #(10k docs)=%d' % (len(doc_dicts)))

/Users/memray/project/kp/OpenNMT-kpg/data/salesforce/10k
Loaded #(10k docs)=11252


##### Sample a paragraph

In [12]:
doc_id = random.randint(0, len(doc_dicts))
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par_id = random.randint(0, len(doc_dicts[doc_id]['paragraph']))
text_to_extract = pars[par_id]
print(doc_id)
print(par_id)
print(text_to_extract)

7705
3
CSX was incorporated in 1978 under Virginia law. In 1980, the Company completed the merger of the Chessie System and Seaboard Coast Line Industries into CSX. The merger allowed the Company to connect northern population centers and Appalachian coal fields to growing southeastern markets. Later, the Company’s acquisition of key portions of Conrail, Inc. ("Conrail") allowed CSXT to link the northeast, including New England and the New York metropolitan area, with Chicago and midwestern markets as well as the growing areas in the Southeast already served by CSXT. This current rail network allows the Company to directly serve every major market in the eastern United States with safe, dependable, environmentally responsible and fuel efficient freight transportation and intermodal service.



### Or load a PROD description

In [2]:
PROD_path = '../data/salesforce/SF_Prod'

prod_dicts = []
for subdir, dirs, files in os.walk(PROD_path):
    for file in files:
        filepath = subdir + os.sep + file
        text = open(filepath, 'r').readlines()
        text = '\n'.join([l.strip() for l in text])
        doc = {'name': file, 'path': filepath, 'text': text}
        prod_dicts.append(doc)

print('Loaded #(PROD docs)=%d' % (len(prod_dicts)))

doc_id = random.randint(0, len(prod_dicts))
doc = prod_dicts[doc_id]
text_to_extract = doc['text']
print(doc_id)
print(doc['name'])
print(text_to_extract)

Loaded #(PROD docs)=20
17
Risk
Anything that threatens a company's ability to meet its target or achieve its financial goals is called business risk
These risks come from a variety of sources, so it's not always the company head or a manager who's to blame
Instead, the risks may come from other sources within the firm or they may be external—from regulations to the overall economy
While a company may not be able to shelter itself from risk completely, there are ways it can help protect itself from the effects of business risk, primarily by adopting a risk management strategy


### Supervised Deep Keyphrase Model

In [6]:
parser = _get_parser()
config_path = '../config/translate/config-rnn-keyphrase.yml'
one2one_ckpt_path = '../models/keyphrase/meng17-one2one-kp20k-topmodels/kp20k-meng17-one2one-rnn-BS128-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Covfalse-Contboth-IF1_step_30000.pt'
one2seq_ckpt_path = '../models/keyphrase/meng17-one2seq-kp20k-topmodels/kp20k-meng17-verbatim_append-rnn-BS64-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Reusetrue-Covtrue-PEfalse-Contboth-IF1_step_50000.pt'
opt = parser.parse_args('-config %s' % (config_path))
setattr(opt, 'models', [one2one_ckpt_path])

translator = build_translator(opt, report_score=False)

In [7]:
scores, predictions = translator.translate(
    src=[text_to_extract],
    tgt=None,
    src_dir=opt.src_dir,
    batch_size=opt.batch_size,
    attn_debug=opt.attn_debug,
    opt=opt
)
print('Paragraph:\n\t'+text_to_extract)
print('Top predictions:')
keyphrases = [kp.strip() for kp in predictions[0] if (not kp.lower().strip() in stoplist) and (kp != '<unk>' )]
for kp_id, kp in enumerate(keyphrases[: min(len(keyphrases), 20)]):
    print('\t%d: %s' % (kp_id+1, kp))

/Users/rui.meng/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


Translating 10/1
Total translation time (s): 0.284127
Average translation time (s): 0.284127
Tokens per second: 7.039106
Paragraph:
	Anything that threatens a company's ability to meet its target or achieve its financial goals is called business risk
These risks come from a variety of sources, so it's not always the company head or a manager who's to blame
Instead, the risks may come from other sources within the firm or they may be external—from regulations to the overall economy
While a company may not be able to shelter itself from risk completely, there are ways it can help protect itself from the effects of business risk, primarily by adopting a risk management strategy
Top predictions:
	1: risk management
	2: business risk
	3: Anything
	4: financial goals
	5: business risk management
	6: risk
	7: company's
	8: business risk,
	9: business
	10: blame
	11: goals
	12: risk management strategy
	13: economy
	14: sources,
	15: risk,
	16: management
	17: completely,
	18: business risk Th

### PKE models

#### TF-IDF

In [3]:
dataset_name = 'SF_Prod'
dataset_path = '../data/salesforce/%s/' % dataset_name
_ = kp_inference.extract_pke(text_to_extract, method='tfidf' , dataset_path=dataset_path,
            df_path=os.path.abspath(dataset_path + '../%s.df.tsv.gz' % dataset_name), top_k=20)

	1: anything (0.0000)
	2: anything that (0.0000)
	3: anything that threatens (0.0000)
	4: that (0.0000)
	5: that threatens (0.0000)
	6: threatens (0.0000)
	7: company (0.0000)
	8: ability (0.0000)
	9: ability to (0.0000)
	10: ability to meet (0.0000)
	11: to meet (0.0000)
	12: to meet its (0.0000)
	13: meet (0.0000)
	14: meet its (0.0000)
	15: meet its target (0.0000)
	16: its (0.0000)
	17: its target (0.0000)
	18: its target or (0.0000)
	19: target (0.0000)
	20: target or (0.0000)


#### YAKE

In [4]:
_ = kp_inference.extract_pke(text_to_extract, method='yake', top_k=20)

	1: called business risk (0.0331)
	2: financial goals (0.0332)
	3: called business (0.0632)
	4: business risk (0.0830)
	5: risk (0.1075)
	6: company (0.1209)
	7: anything (0.1383)
	8: ability (0.1383)
	9: risks may come (0.1415)
	10: business (0.1657)
	11: may (0.1765)
	12: threatens (0.1793)
	13: meet (0.1793)
	14: target (0.1793)
	15: achieve (0.1793)
	16: financial (0.1793)
	17: goals (0.1793)
	18: called (0.1793)
	19: risk management strategy (0.1844)
	20: anything that threatens (0.1851)


#### TextRank

In [5]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: risk management (0.1901)
	2: company head (0.1464)
	3: financial goals (0.1464)
	4: risk (0.0950)
	5: sources (0.0732)
	6: company (0.0732)
